**Thông tin nhóm: Nhóm 2**

**20120083 - Nguyễn Trọng Hiếu**

**20120105 - Lê Hoàng Huy**


### 1. Application description
**What is your chosen application?**

Our chosen application: Thực hiện và tối ưu hóa forward-pass của convolutional layer bằng CUDA.

<li> Fashion MNIST </li>

* Input: Ảnh được lưu dưới dạng Eigen::Matrix, số lượng cột là số lượng ảnh, số lượng hàng là thông tin của một ảnh

<div style="display: flex">

  <div style="width: 30%; text-align: center;">

| image 0 | image 1 | image 2 |
|:-------:|:-------:|:-------:|
| 1<br>6<br>2<br>1<br>2<br>5<br>6<br>0<br>4 | 3<br>1<br>6<br>2<br>8<br>9<br>7<br>4<br>0 | 7<br>1<br>1<br>4<br>1<br>8<br>5<br>0<br>7 |
  </div>
  <div style="width: 30%; color = black; text-align: center;">

*Image1*

        1 1 6
        6 2 0
        2 5 4
  </div>
  <div style="width: 30%; color = black; text-align: center;">
  
*Matrix input*

        1 1 6
        6 2 0
        2 5 4
        1 2 4
        2 8 1
        5 9 8
        6 7 5
        0 4 0
        4 0 7
  </div>
</div
<br>

*<li>* convolutional layer</li>

* Input: Một batch các ảnh dưới dạng Matrix, số lượng cột là số lượng ảnh trong batch, số lượng hàng là thông tin của một ảnh được thể nối các giá trị theo cột như mô tả bên trên.

* Output: Số lượng ảnh đầu ra với số hàng là số lượng ảnh và số cột là height_out*width_out

* Use cases: khó để lấy vị dụ ảnh thông qua C/C++

**Does it need to speed up? Why**

Do thời gian chạy trên cpu lâu và có khả năng song song hóa trên forward-pass của convolutional layer.


### 2. Sequential implementation
**Design: Describe steps to go from input to output (don’t show code)**

Dựa vào mô hình base <a href='https://github.com/iamhankai/mini-dnn-cpp'>mini-dnn-cpp</a> tiến hành 
* Xây dựng kiến trúc <a href='https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide'>LeNet-5</a>
* Chạy trên bộ dữ liệu <a href='https://github.com/zalandoresearch/fashion-mnist'>Fashion MNIST</a>
* Tiến hành chạy thử với <a href='https://colab.research.google.com/drive/1pwaG9S3LJyYteGH4P4GQD8R0wLte-BNy?authuser=1#scrollTo=uL24K0K1_m88'>cpu trên  google-colab</a>

**Evaluate:**
* Describe your experiment setup

* Run the code to see results?

*_Kết quả được thể hiện trên_*

* Does it run correctly?

*_Kết quả chạy đúng mong muốn của thiết lập ban đầu._*

### 3. Parallel implementation
**Analyze: Which steps do you parallelize? Why these steps?**

Thực hiện song song trên forward-pass của convolutional layer.
Nhận xét tính khả thi trong việc tận dụng tính toán song song của GPU để tiết kiệm thời gian.

**Design: How do you parallelize? (don’t show code)**

**Evaluate:**

* Describe your experiment setup
* Runthe code to see results
* Doesit run correctly & faster? If not, do you know why?

### 4. Parallel implementation + optimization

**Version 1**

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

**Design: How do you optimize? (don’t show code)**

**Evaluate:**

* Describe your experiment setup
* Runthe code to see results
* Doesit run correctly & faster? If not, do you know why?

**Version 2**

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

**Design: How do you optimize? (don’t show code)**

**Evaluate:**

* Describe your experiment setup
* Runthe code to see results
* Doesit run correctly & faster? If not, do you know why?

### 5. Reflection
* Eachmember: What difficulties have you encountered?
* Eachmember: What have you learned?
* Your team: If you had more time, what would you do?